In [1]:
import os
import re
import textwrap
import warnings
import pyperclip
import soundfile as sf
from pynput import keyboard
from plyer import notification
from playsound import playsound
from num2words import num2words
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

# this is 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.system("export CUDA_VISIBLE_DEVICES=\"\"")

0

** SUMMERIZATION models

In [2]:
# # facebook/bart-large-cnn
# from transformers import pipeline
# summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
# def facebook_bart_large_cnn(text):
#     summary = summarizer(text, max_length=20 if len(text.split())<=50 else 70, min_length=10, do_sample=False)
#     return summary[0]['summary_text']

In [3]:
# google/bigbird-pegasus-large-bigpatent
summarytokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-bigpatent")
summarymodel = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-bigpatent", attention_type="original_full")
def google_bigbird_pegasus_large_bigpatent(text):
    inputs = summarytokenizer(text, return_tensors='pt')
    summaryprediction = summarymodel.generate(**inputs)
    thesummary = summarytokenizer.batch_decode(summaryprediction)
    return thesummary[0].replace('<s>', '').replace('</s>', '').strip()

** Text To Speech models

In [4]:
# # microsoft/speecht5_tts
# import torch
# from datasets import load_dataset
# from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
# processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
# model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
# vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
# embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
# speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# def microsoft_speecht5_tts(text):
#     inputs = processor(text=text, return_tensors="pt")
#     return model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder), 22000

In [5]:
# facebook/fastspeech2-en-ljspeech
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
fairseqmodels, fairseqcfg, fairseqtask = load_model_ensemble_and_task_from_hf_hub(
    "facebook/fastspeech2-en-ljspeech",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
fairseqmodel = fairseqmodels[0].to('cpu')
TTSHubInterface.update_cfg_with_data_cfg(fairseqcfg, fairseqtask.data_cfg)
generator = fairseqtask.build_generator(fairseqmodels, fairseqcfg)

def facebook_fastspeech2_en_ljspeech(text):
    sample = TTSHubInterface.get_model_input(fairseqtask, text)
    return TTSHubInterface.get_prediction(fairseqtask, fairseqmodel, generator, sample)

2024-02-12 17:07:01.788135: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 17:07:01.788168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 17:07:01.789143: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-12 17:07:03.226350: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-02-12 17:07:04 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

2024-02-12 17:07:05 | INFO | fairseq.tasks.speech_to_text | dictionary size (vocab.txt): 75
/home/alireza/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
2024-02-12 17:07:07 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/alireza/.cache/fairseq/models--facebook--fastspeech2-en-ljspeech/snapshots/a3e3e5e2e62bb7ca7514b11aa469e9c5b01a20bf/hifigan.bin


In [6]:
def show_notification(title, message):
    notification.notify(
        title=title,
        message=message,
        timeout=10
    )

In [7]:
def summerize(text):
    print(len(text.split()))
    print(textwrap.fill(text, 80))

    # return facebook_bart_large_cnn(text)
    return google_bigbird_pegasus_large_bigpatent(text)

In [8]:
def readoutload(text):
    # wav, rate = microsoft_speecht5_tts(text)
    wav, rate = facebook_fastspeech2_en_ljspeech(text)
    
    sf.write("nowgeneratedspeechforstudy.wav", wav, samplerate=rate)
    show_notification("summary", text)
    playsound("nowgeneratedspeechforstudy.wav")

In [9]:
def convert_numbers_to_text(text):
    # Regular expression pattern to match numbers
    pattern = r'\b\d+\b'
    
    def replace(match):
        number = int(match.group())
        return num2words(number)
    
    # Replace numbers in the text with their textual representation
    converted_text = re.sub(pattern, replace, text)

    return converted_text

def preprocesstext(text):
    text = text.strip()
    text = text.replace('-\n', '')
    text = text.replace('\n', ' ')
    
    text = convert_numbers_to_text(text)
    # text = text.replace(".", ",").replace("!", ",").replace("?", ",").replace(":", ",").replace(";", ",")
    text = text.replace("(",',').replace(")",',').replace("[",',').replace("]",',').replace("{",',').replace("}",',')
    text = text.replace('"',',').replace("“",',').replace("”",',')
    text = text.replace("-",' ').replace("_",' ').replace("—",' ').replace("–",' ').replace("…",' ')
    
    return text

In [10]:
prev_originaltext = ''
def generatebytext(originaltext, mode):
    global prev_originaltext
    originaltext = preprocesstext(originaltext)

    if mode == 'stts':
        if originaltext != prev_originaltext:
            
            thesummary = originaltext if len(originaltext.split())<=10 else summerize(originaltext)
            thesummarysplitted = [substr for substr in re.split(r"[.!?;:]", thesummary) if substr]
            print(thesummarysplitted)
            for tmptext in thesummarysplitted: readoutload(tmptext)
            prev_originaltext = thesummary
    elif mode == 'tts':
        if originaltext != prev_originaltext:
            thesummarysplitted = [substr for substr in re.split(r"[.!?;:]", originaltext) if substr]
            print(thesummarysplitted)
            for tmptext in thesummarysplitted: readoutload(tmptext)
            prev_originaltext = originaltext


## test

In [18]:
generatebytext("anti-cancer drugs. Potent cytokines and monoclonal antibodies", "tts")

['anti cancer drugs', ' Potent cytokines and monoclonal antibodies']


In [ ]:
generatebytext("a", "stts")

In [22]:
import json
 
# Opening JSON file
f = open('./output.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)

In [29]:
generatebytext(data['Page'][0]['TextLine'][2]["text"], "tts")

['directed against cell surface associated structures are already promi ']
